In [1]:
import cv2
import numpy as np

In [2]:
webcam = True
path = '1.jpg'
cap = cv2.VideoCapture(0) # webcam 1
cap.set(10,160) # brightness
cap.set(3,1920) # width
cap.set(4,1080) # height
scale = 3
wP = 210 * scale
hP = 297 * scale

In [3]:
def getContours(img, cThr=[100,100], minArea=1000, filter=0, showCanny=False, showThre=False, draw=False):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5,5), 1)
    imgCanny = cv2.Canny(imgBlur, cThr[0], cThr[1])
    if showCanny: cv2.imshow('Canny', imgCanny)
    
    kernel = np.ones((5,5))
    imgDial = cv2.dilate(imgCanny, kernel, iterations=3)
    imgThre = cv2.erode(imgDial, kernel, iterations=2)
    if showThre: cv2.imshow('Thre', imgThre)
    
    contours, hiearchy = cv2.findContours(imgThre, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    finalCountours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > minArea:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02*peri, True)
            bbox = cv2.boundingRect(approx)
            if filter > 0:
                if len(approx) == filter:
                    finalCountours.append([len(approx), area, approx, bbox, contour])
            else:
                finalCountours.append([len(approx), area, approx, bbox, contour])
    
    finalCountours = sorted(finalCountours, key=lambda x:x[1], reverse=True)
    
    if draw:
        for contour in finalCountours:
            cv2.drawContours(img, contour[4], -1, (0,0,255), 3)
    
    return img, finalCountours

In [4]:
def reOrder(points):
    resultPoints = np.zeros_like(points)
    points = points.reshape((4,2))
    add = points.sum(1)
    resultPoints[0] = points[np.argmin(add)]
    resultPoints[3] = points[np.argmax(add)]
    diff = np.diff(points, axis=1)
    resultPoints[1] = points[np.argmin(diff)]
    resultPoints[2] = points[np.argmax(diff)]
    return resultPoints

In [5]:
def warpImg(img, points, width, height, pad=20):
    points = reOrder(points)
    pts1 = np.float32(points)
    pts2 = np.float32([[0,0], [width,0], [0,height], [width,height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (width,height))
    imgWarp = imgWarp[pad:imgWarp.shape[0]-pad, pad:imgWarp.shape[1]-pad]
    return imgWarp

In [6]:
def findDistance(pts1,pts2):
    return ((pts2[0]-pts1[0])**2 + (pts2[1]-pts1[1])**2)**0.5

In [7]:
while True:
    if webcam: success, frame = cap.read()
    else: frame = cv2.imread(path)
    
    frame, finalCountours = getContours(frame,minArea=50000, filter=4)
    
    if len(finalCountours) != 0:
        biggest = finalCountours[0][2]
        imgWarp = warpImg(frame, biggest, wP, hP)
#         cv2.imshow('A4', imgWarp)
    
        innerFrame, countours2 = getContours(imgWarp, minArea=2000, filter=4, cThr=[50,50])
#         cv2.imshow('Inner Contour', innerFrame)
        
        if len(countours2) != 0:
            for obj in countours2:
                cv2.polylines(innerFrame, [obj[2]], True, (0,255,0), 2)
                nPoints = reOrder(obj[2])
                nW = round(findDistance(nPoints[0,0]//scale, nPoints[1][0]//scale)/10,1)
                nH = round(findDistance(nPoints[0,0]//scale, nPoints[2][0]//scale)/10,1)
                
                cv2.arrowedLine(innerFrame, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[1][0][0], nPoints[1][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05)
                cv2.arrowedLine(innerFrame, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[2][0][0], nPoints[2][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05)
                x, y, w, h = obj[3]
                cv2.putText(innerFrame, '{}cm'.format(nW), (x + 30, y - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 255), 2)
                cv2.putText(innerFrame, '{}cm'.format(nH), (x - 70, y + h // 2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 255), 2)
        
        cv2.imshow('Draw Line', innerFrame)
#     img = cv2.resize(frame, (0,0), None, 0.5, 0.5)
#     cv2.imshow('Original', frame)
#     cv2.imshow('Draw Line', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()